# Prompting FLAN-T5

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## Load the tokenizer and model

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Summary prompt

In [7]:
prompt = "summarize: Like rock-climbing, snowboarding is an extreme sport. Whether I am snowboarding the bluffs of Wisconsin or the Mountains of Colorado, I get this intense and overwhelming rush of adrenaline that runs through my body. When my friends and I take the slopes, we snowboard from the early morning hours until closing time. Unfortunately, there are times when my friends and I cannot go snowboarding because of undesired weather conditions. Icy surfaces, for example, are the worst conditions, which make control and precision difficult. However, when the conditions are right, the only place to find me is on the slopes. On the beautiful days, when the sun is shining brightly and the snow is fluffy as cotton, this becomes our playground. Setting up large jumps, racing through the halfpipe and grasping the fastest speeds possible make for exciting days while on the slopes. I remember when I first began this sport. It was no easy task. There were many disadvantages in learning how to snowboard. I took a severe beating every time I went. One time I broke my arm and another time I broke my tailbone, but that did not stop me. I loved the thrill of snowboarding and with the support of my friends and my dedication to the sport, this is how snowboarding became a part of my life."
print(len(prompt))
# Generate inputs
inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True, padding=True)

# Generate outputs
outputs = model.generate(inputs["input_ids"], max_length=300, num_beams=5, early_stopping=True)

# Decode and print the summary
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


1288
The author describes snowboarding as an extreme sport.


### Summary with Few-Shot learning

In [14]:
# Few-shot examples
few_shot_examples = [
    "Summarise: 'The quick brown fox jumps over the lazy dog. The dog was not amused by the fox's antics.' Summary: 'The fox jumped over the dog, who was not happy.'",
    "Summarise: 'The rain in Spain stays mainly in the plain. It was a wet and rainy season in the Spanish plains.' Summary: 'It was a rainy season in the Spanish plains.'"
]

# New prompt to summarize
prompt = "Summarise: 'Like rock-climbing, snowboarding is an extreme sport. Whether I am snowboarding the bluffs of Wisconsin or the Mountains of Colorado.'"

# Combine examples and prompt
combined_prompt = "\n\n".join(few_shot_examples + [prompt])

# Generate inputs
inputs = tokenizer(combined_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)

# Generate outputs
outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)

# Decode and print the summary
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Summary: 'Snowboarding is an extreme sport,' he said.


## Translation prompt

In [8]:
# Define the translation prompt
translation_prompt = "translate English to Danish: Nile is the longest river in the world."

# Prepare inputs and generate outputs
translation_inputs = tokenizer(translation_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)
translation_outputs = model.generate(translation_inputs["input_ids"], max_length=40, num_beams=5, early_stopping=True)

# Decode and display the translation
print(tokenizer.decode(translation_outputs[0], skip_special_tokens=True))

Nile är den längre rdet i världen.


### Translation with Few-Shot learning

In [17]:

# Here, we provide a few examples along with their translations to help the model understand the pattern.

# Few-shot examples
few_shot_examples = [
    "translate English to Danish: 'Mount Everest is the highest mountain in the world.' Translation: 'Mount Everest er det højeste bjerg i verden.'",
    "translate English to Danish: 'The Amazon is the largest rainforest on Earth.' Translation: 'Amazonas er den største regnskov på Jorden.'",
    "translate English to Danish: 'The Danube is the second longest river in Europe.' Translation: 'Donau er den næstlængste flod i Europa.'"
]

# New prompt to translate
translation_prompt = "translate English to Danish: 'Nile is the longest river in the world.'"

# Combine examples and prompt
combined_translation_prompt = "\n\n".join(few_shot_examples + [translation_prompt])

# Prepare inputs and generate outputs
translation_inputs = tokenizer(combined_translation_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)
translation_outputs = model.generate(translation_inputs["input_ids"], max_length=40, num_beams=5, early_stopping=True)

# Decode and display the translation
print(tokenizer.decode(translation_outputs[0], skip_special_tokens=True))


Translation: 'Nile er den nstlngste flod i Europa.'


## Q&A

In [9]:
# Define the context and question
context_question = "The Great Wall of China is over 13,000 miles long. question: How long is the Great Wall of China?"

# Generate inputs
question_inputs = tokenizer(context_question, return_tensors="pt", max_length=512, truncation=True, padding=True)

# Generate outputs
question_outputs = model.generate(question_inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)

# Decode and print the answer
print(tokenizer.decode(question_outputs[0], skip_special_tokens=True))


over 13,000 miles


### Q&A with In-context learning

In [19]:
# Few-shot examples
few_shot_examples = [
    "The Great Wall of China is over 13,000 miles long. question: 'How long is the Great Wall of China?' answer: 'The Great Wall of China is over 13,000 miles long.'",
    "The capital of France is Paris. question: 'What is the capital of France?' answer: 'The capital of France is Paris.'"
]

# New context and question
context_question = "Mount Everest is the highest mountain in the world. question: 'What is the highest mountain in the world?'"

# Combine examples and prompt
combined_qa_prompt = "\n\n".join(few_shot_examples + [context_question])

# Generate inputs
question_inputs = tokenizer(combined_qa_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)

# Generate outputs
question_outputs = model.generate(question_inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)

# Decode and print the answer
print(tokenizer.decode(question_outputs[0], skip_special_tokens=True))


Answer: 'Mount Everest
